### Declaração das bibliotecas

In [1]:
import math
import json

#### Abre o json com as informações das salas (dimensão)

In [2]:
with open('rooms.json', 'r') as json_file:
    rooms = json.load(json_file)

#### Calcula a quantidade de paredes por intersecção de retas.

In [10]:
class Wall:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2

    def interceptPoint(self, x1, y1, x2, y2):
        p0_x = self.x1
        p0_y = self.y1
        p1_x = self.x2
        p1_y = self.y2

        p2_x = x1
        p2_y = y1
        p3_x = x2
        p3_y = y2
        
        s1_x = p1_x - p0_x
        s1_y = p1_y - p0_y
        s2_x = p3_x - p2_x
        s2_y = p3_y - p2_y

        s = (-s1_y * (p0_x - p2_x) + s1_x * (p0_y - p2_y)) / (-s2_x * s1_y + s1_x * s2_y)
        t = ( s2_x * (p0_y - p2_y) - s2_y * (p0_x - p2_x)) / (-s2_x * s1_y + s1_x * s2_y)

        if (s >= 0) and (s <= 1) and (t >= 0) and (t <= 1):
            intX = p0_x + (t * s1_x)
            intY = p0_y + (t * s1_y)
            
            return [intX, intY];

        return -1

#### Calcula o tamanho de todas as paredes do cenário (top, bottom, right, left)

In [7]:
walls = []

for r in rooms['rooms']:
    room = rooms['rooms'][r]
    
    if room['type'] == 'room':
        #print(room)
        xWidth = room['x']+room['width']
        yHeight = room['y']+room['height']

        walls.append(Wall(room['x'], room['y'], xWidth   , room['y'] ));
        #print('       top:  [',room['x'],',', room['y'],',', xWidth, ',', room['y'], ']')
        
        walls.append(Wall(xWidth   , room['y'], xWidth   , yHeight   ));
        #print('       right:[',xWidth,',', room['y'],',', xWidth, ',', yHeight, ']')
        
        walls.append(Wall(xWidth   , yHeight  , room['x'], yHeight   ));
        #print('      bottom:[',xWidth,',',yHeight,',', room['x'], ',', yHeight, ']')
        
        walls.append(Wall(room['x'], yHeight  , room['x'], room['y'] ));
        #print('      left  :[',room['x'],',',yHeight,',', room['x'], ',', room['y'], ']\n')

#### Realiza o cálculo de paredes entre dois pontos usando as funções acima. O primeiro ponto é a posição do beacon, o segundo é  a posição do ponto.

In [5]:
def intercept(x1, y1, x2, y2):

    if (x1 == x2) and (y1 == y2):
        return []

    allInterceptions = []
    
    for wall in walls:
        intercept = wall.interceptPoint(x1, y1, x2, y2)

        if intercept != -1:
            allInterceptions.append(intercept);
    
    result = []

    for testPoint in allInterceptions:
        foundClose = 'false'

        for resultPoint in result:
            dist = math.sqrt(math.pow(testPoint[0] - resultPoint[0], 2) + math.pow(testPoint[1]-resultPoint[1], 2))

            if dist < 5:
                foundClose = 'true'
                break

        if foundClose == 'false':
            result.append(testPoint)

    return result

In [6]:
p1 = [397,416]
p2 = [2645,607]

res = len(intercept(p1[0],p1[1],p2[0],p2[1]))
print('Paredes:', res)

Paredes: 5
